In this walkthrough we will build a full text search capable application by incrementally building the features to be exposed via a simple REST API.

## Install Prerequisites

In [1]:
! pip install "pymongo[srv]"


You should consider upgrading via the '/Users/tom.mccarthy/Projects/tommcc-demos/.venv/bin/python -m pip install --upgrade pip' command.


In [2]:
# Just making sure everything prints cleanly

from pygments.style import Style
from pygments.token import Token
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import Terminal256Formatter


class MyStyle(Style):
    styles = {
        Token.String: 'ansigreen',
        Token.Literal: 'ansibrightyellow',
        Token.Keyword: 'ansimagenta',
        Token.Operator: 'ansibrightmagenta'
    }

def pp(doc):
    formatted_json = json.dumps(json.loads(json_util.dumps(doc)), indent=4)
    colorful_json = highlight(formatted_json, JsonLexer(), Terminal256Formatter(style=MyStyle))
    print(colorful_json)

Add parent to sys path to load config

In [3]:
import sys
import os

sys.path.append("..")

In [4]:
import pymongo
from bson import json_util
from config import mongo_uri
import json
    
conn = pymongo.MongoClient(mongo_uri)

movies_collection = conn['sample_mflix']['movies']

## Basic Search

Run a simple text search.

In [5]:
pipeline = [
    {
        '$search': {
            'text': {
                'query': "fight",
                'path': "title"
            }
        }
    },
    {
        '$project': {
            'title':1,
            '_id':0,
            'score': {
                '$meta': 'searchScore'
            }
        }
    }
]
docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fighting",
        "score": 4.2303690910339355
    },
    {
        "title": "Fight, Zatoichi, Fight",
        "score": 4.106432914733887
    },
    {
        "title": "Fight, Zatoichi, Fight",
        "score": 4.106432914733887
    },
    {
        "title": "The Fighting Lady",
        "score": 3.4211220741271973
    },
    {
        "title": "Fight Club",
        "score": 3.4211220741271973
    },
    {
        "title": "The Fighting Temptations",
        "score": 3.4211220741271973
    },
    {
        "title": "Street Fight",
        "score": 3.4211220741271973
    },
    {
        "title": "Girl Fight",
        "score": 3.4211220741271973
    },
    {
        "title": "A Fighting Man",
        "score": 3.4211220741271973
    },
    {
        "title": "Fight Back to School",
        "score": 2.8717684745788574
    },
    {
        "title": "Why We Fight",
        "score": 2.8717684745788574
    },
    {
        "title": "Love at First Fight",
        "scor

## Fuzzy

Often referred to as approximate string matching, fuzzy matching is a technique of finding strings that match a pattern approximately rather than exactly. It's common use case is when there are common mispellings, when users make errors ("fat fingering"), etc.

**maxEdits** uses the Levenshtein distance, which is the difference between two string sequences.

In [6]:
pipeline = [
    {
        '$search': {
            'text': {
                'query': "might cub",
                'path': "title",
                'fuzzy':{
                    'maxEdits':2
                }
            }
        }
    },
    {
        '$project': {
            'title':1,
            '_id':0,
            'score': {
                '$meta': 'searchScore'
            }
        }
    }
]
docs = movies_collection.aggregate(pipeline)
pp(docs)


[
    {
        "title": "Fight Club",
        "score": 3.7854979038238525
    },
    {
        "title": "Cub",
        "score": 3.6564555168151855
    },
    {
        "title": "Knight of Cups",
        "score": 3.331955909729004
    },
    {
        "title": "Fright Night",
        "score": 3.1747939586639404
    },
    {
        "title": "Fright Night",
        "score": 3.1747939586639404
    },
    {
        "title": "Eight Crazy Nights",
        "score": 3.045708179473877
    },
    {
        "title": "Friday Night Lights",
        "score": 3.045708179473877
    },
    {
        "title": "The High and the Mighty",
        "score": 2.9480228424072266
    },
    {
        "title": "Monster High: Friday Night Frights",
        "score": 2.7375903129577637
    },
    {
        "title": "Fight for Your Right Revisited",
        "score": 2.62430477142334
    },
    {
        "title": "Eight Miles High",
        "score": 2.474637985229492
    },
    {
        "title": "The Club",
        

## Highlighting

Add a relevance score and hit highlights to the results.

In [7]:
pipeline = [
        {
            '$search': {
                'text': {
                    'query': "fight",
                    'path': "title"
                },
                # text highlighting
                'highlight': { "path": "title" }
            }
        }, {
            '$project': {
                'title':1,
                '_id':0,
                'score': {
                    '$meta': 'searchScore'
                },
                'highlights': {"$meta": "searchHighlights"},
                'score': {
                    '$meta': 'searchScore'
                }
            }
        }
    ]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fighting",
        "score": 4.2303690910339355,
        "highlights": [
            {
                "score": 1.408543348312378,
                "path": "title",
                "texts": [
                    {
                        "value": "Fighting",
                        "type": "hit"
                    }
                ]
            }
        ]
    },
    {
        "title": "Fight, Zatoichi, Fight",
        "score": 4.106432914733887,
        "highlights": [
            {
                "score": 1.1319897174835205,
                "path": "title",
                "texts": [
                    {
                        "value": "Fight",
                        "type": "hit"
                    },
                    {
                        "value": ", Zatoichi, ",
                        "type": "text"
                    },
                    {
                        "value": "Fight",
                        "type": "hit"
                    

## Autocomplete

Edit the "query" field to try different searches

In [8]:
pipeline = [
    {
        '$search': {
            'autocomplete': {
                'query': "fl",
                'path': "title"
            }
        }
    },
    {
        '$project': {
            'title': 1,
            'score': {
                    '$meta': 'searchScore'
                },
            '_id': 0
        }
    },
    {
        '$limit': 5
    },
    {
        '$sort': {'score': -1}
    }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "The Flim-Flam Man",
        "score": 4.277243614196777
    },
    {
        "title": "Flower Drum Song",
        "score": 4.10808801651001
    },
    {
        "title": "Flatfoot in Africa",
        "score": 4.10808801651001
    },
    {
        "title": "Flodder in Amerika!",
        "score": 4.10808801651001
    },
    {
        "title": "Flesh and Bone",
        "score": 4.10808801651001
    }
]



# Compound

Compound queries grant the ability to add multiple conditions to your search query. Each element is a clause, where you can embed sub-queries.

In [10]:
pipeline = [
    {
        "$search": {
            "compound": {
                "must": [{"text": {"query": "Fight", "path": "title"}}],
                "mustNot": [
                    {"text": {"query": "Zatoichi", "path": "title"}},
                    {"text": {"query": "Club", "path": "title"}},
                ],
            }
        }
    },
    {"$project": {"title": 1, "score": {"$meta": "searchScore"}, "_id": 0}},
    {"$limit": 5},
    {"$sort": {"score": -1}},
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fighting",
        "score": 4.2303690910339355
    },
    {
        "title": "The Fighting Lady",
        "score": 3.4211220741271973
    },
    {
        "title": "The Fighting Temptations",
        "score": 3.4211220741271973
    },
    {
        "title": "Street Fight",
        "score": 3.4211220741271973
    },
    {
        "title": "Girl Fight",
        "score": 3.4211220741271973
    }
]



# Facets

In [11]:
from dateutil import parser

pipeline = [
 {
   "$searchMeta": {
     "count": {"type": "total"},
     "facet": {
       "operator": {
         "range": {
           "path": "released",
           "gte": parser.parse("2000-01-01T00:00:00.000Z"),
           "lte": parser.parse("2015-01-31T00:00:00.000Z")
         }
       },
       "facets": {
         "directorsFacet": {
           "type": "string",
           "path": "directors",
           "numBuckets" : 5
         },
         "yearFacet" : {
           "type" : "number",
           "path" : "year",
           "boundaries" : [2000,2005,2010,2015]
         },
         "genresFacet": {
            "type": "string",
            "path": "genres",
            "numBuckets": 5
          }
       }
     }
   }
 }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "count": {
            "total": 13064
        },
        "facet": {
            "genresFacet": {
                "buckets": [
                    {
                        "_id": "Drama",
                        "count": 7387
                    },
                    {
                        "_id": "Comedy",
                        "count": 3795
                    },
                    {
                        "_id": "Romance",
                        "count": 1863
                    },
                    {
                        "_id": "Thriller",
                        "count": 1592
                    },
                    {
                        "_id": "Documentary",
                        "count": 1573
                    }
                ]
            },
            "yearFacet": {
                "buckets": [
                    {
                        "_id": 2000,
                        "count": 3283
                    },
                    {
 

# Synonyms

Requires index set up with synonyms for title field. 

In [12]:
pipeline = [
  {
    '$search': {
      'text': {
        'path': 'title',
        'query': 'automobile',
        'synonyms': 'transportSynonyms'
      }
    }
  }, 
  {
    '$limit': 10
  }, 
  {
    '$project': {
      '_id': 0,
      'title': 1,
      'score': {
        '$meta': 'searchScore'
      }
    }
  }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Cars",
        "score": 4.197734832763672
    },
    {
        "title": "Planes, Trains & Automobiles",
        "score": 3.8511905670166016
    },
    {
        "title": "Car Wash",
        "score": 3.39473032951355
    },
    {
        "title": "Used Cars",
        "score": 3.39473032951355
    },
    {
        "title": "Blue Car",
        "score": 3.39473032951355
    },
    {
        "title": "Cars 2",
        "score": 3.39473032951355
    },
    {
        "title": "Stealing Cars",
        "score": 3.39473032951355
    },
    {
        "title": "Cop Car",
        "score": 3.39473032951355
    },
    {
        "title": "The Cars That Eat People",
        "score": 2.8496146202087402
    },
    {
        "title": "Khrustalyov, My Car!",
        "score": 2.8496146202087402
    }
]



In [13]:
pipeline = [
    {
        '$search': {
            'compound': {
                'should': [
                    {
                        'text': {
                            'path': 'title',
                            'query': 'boat',
                            'synonyms': 'transportSynonyms'
                        }
                    }, {
                        'text': {
                            'path': 'title',
                            'query': 'hat',
                            'synonyms': 'attireSynonyms'
                        }
                    }
                ]
            }
        }
    }, {
        '$limit': 10
    }, {
        '$project': {
            '_id': 0,
            'title': 1,
            'score': {
                '$meta': 'searchScore'
            }
        }
    }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fedora",
        "score": 5.673145294189453
    },
    {
        "title": "Vessel",
        "score": 5.373150825500488
    },
    {
        "title": "Boats",
        "score": 4.589139938354492
    },
    {
        "title": "And the Ship Sails On",
        "score": 4.3452959060668945
    },
    {
        "title": "Broken Vessels",
        "score": 4.3452959060668945
    },
    {
        "title": "Sailing to Paradise",
        "score": 4.3452959060668945
    },
    {
        "title": "Top Hat",
        "score": 4.066137313842773
    },
    {
        "title": "A Hatful of Rain",
        "score": 4.066137313842773
    },
    {
        "title": "Boat People",
        "score": 3.711261749267578
    },
    {
        "title": "Boat Trip",
        "score": 3.711261749267578
    }
]



In [42]:
from bson import ObjectId

pipeline = [
    {"$match": {"title": "The Godfather"}},
    {"$project": {"genres": 1, "plot": 1}},
]

doc = list(movies_collection.aggregate(pipeline))[0]

pipeline = [
    {
        "$search": {
            "compound": {
                "must": [{"moreLikeThis": {"like": doc}}],
                "mustNot": [
                    {
                        "equals": {
                            "path": "_id",
                            "value": doc["_id"],
                        }
                    }
                ],
            }
        }
    },
    {"$project": {"score": {"$meta": "searchScore"}, "title": 1}},
]


docs = movies_collection.aggregate(pipeline)

pp(docs)

[
    {
        "_id": {
            "$oid": "573a13b1f29313caabd36962"
        },
        "title": "Romanzo Criminale",
        "score": 7.608759880065918
    },
    {
        "_id": {
            "$oid": "573a1398f29313caabce92e8"
        },
        "title": "Night Patrol",
        "score": 6.961604595184326
    },
    {
        "_id": {
            "$oid": "573a1398f29313caabce9d9c"
        },
        "title": "Anguish",
        "score": 6.427550315856934
    },
    {
        "_id": {
            "$oid": "573a13b5f29313caabd4397a"
        },
        "title": "Curse of the Golden Flower",
        "score": 6.113086700439453
    },
    {
        "_id": {
            "$oid": "573a13ecf29313caabdd1e08"
        },
        "title": "The White Haired Witch of Lunar Kingdom",
        "score": 5.955801963806152
    },
    {
        "_id": {
            "$oid": "573a13dff29313caabdb9293"
        },
        "title": "Force of Execution",
        "score": 5.720892429351807
    },
    {
        "

# More like this

In [14]:
# conn.close()